<a href="https://colab.research.google.com/github/samarkk/spark_python/blob/master/pyspark_project_sunlife_oct_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=964d6440097f2f6c775a92bd61091d6dd3a079b4945b82c0590b3470c4098042
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("ColabApp").getOrCreate()
spark.version

'3.5.0'

In [3]:
cm_data_loc = '/content/drive/MyDrive/sunlife/cm_sample_data'
cm_df = spark.read.option('inferSchema', True) \
.option('header', True) \
.csv(cm_data_loc)

In [4]:
cm_df.show(2, False)

+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
|SYMBOL    |SERIES|OPEN |HIGH |LOW  |CLOSE|LAST |PREVCLOSE|TOTTRDQTY|TOTTRDVAL|TIMESTAMP  |TOTALTRADES|ISIN        |_c13|
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
|182D070324|TB    |96.0 |96.0 |96.0 |96.0 |96.0 |96.62    |100      |9600.0   |11-SEP-2023|1          |IN002023Y243|NULL|
|182D071223|TB    |97.87|97.87|97.87|97.87|97.87|97.87    |100      |9787.0   |11-SEP-2023|1          |IN002023Y102|NULL|
+----------+------+-----+-----+-----+-----+-----+---------+---------+---------+-----------+-----------+------------+----+
only showing top 2 rows



In [5]:
nifty_data_loc ='/content/drive/MyDrive/sunlife/nse2023.csv'
nifty_df = spark.read.option('inferSchema', True) \
.option('header', True) \
.csv(nifty_data_loc)

In [6]:
nifty_df.show(2)

+-----------+-------+--------+--------+--------+
|       Date|   Open|    High|     Low|   Close|
+-----------+-------+--------+--------+--------+
|29-Sep-2023|19581.2|19726.25|19551.05| 19638.3|
|28-Sep-2023|19761.8|19766.65| 19492.1|19523.55|
+-----------+-------+--------+--------+--------+
only showing top 2 rows



In [7]:
fo_df_loc = '/content/drive/MyDrive/sunlife/fo_sample_data'
fo_df = spark.read.option('inferSchema', True) \
.option('header', True) \
.csv(fo_df_loc)

In [8]:
fo_df.show(2)

+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|   HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|_c15|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
|    FUTIDX|BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05|46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|NULL|
|    FUTIDX|BANKNIFTY|31-Aug-2023|      0.0|        XX| 46419.9|46486.1| 45829.7|45986.45| 45986.45|   181369|1254975.82| 1884000|   344535|27-JUL-2023|NULL|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+----+
only showing top 2 rows



In [9]:
fodf = fo_df.drop("_c15")

In [10]:
fodf.createOrReplaceTempView('fotbl')

In [11]:
spark.sql('select * from fotbl limit 2').show()

+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|   SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|   HIGH|     LOW|   CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX|BANKNIFTY|27-Jul-2023|      0.0|        XX|46201.05|46500.0|45541.65|45654.75|  45679.3|   203561|1400924.45|  604725|  -543540|27-JUL-2023|
|    FUTIDX|BANKNIFTY|31-Aug-2023|      0.0|        XX| 46419.9|46486.1| 45829.7|45986.45| 45986.45|   181369|1254975.82| 1884000|   344535|27-JUL-2023|
+----------+---------+-----------+---------+----------+--------+-------+--------+--------+---------+---------+----------+--------+---------+-----------+



In [12]:
fodf_nifty = fodf.filter(fodf['SYMBOL'] == 'NIFTY')
fodf_nifty.show(2)

+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|    HIGH|     LOW|  CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX| NIFTY|27-Jul-2023|      0.0|        XX|19840.2|19872.35| 19600.0|19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|
|    FUTIDX| NIFTY|31-Aug-2023|      0.0|        XX|19951.1| 19988.4|19751.05|19811.4|  19811.4|   185376| 1840717.8|11728350|  3456250|27-JUL-2023|
+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
only showing top 2 rows



In [13]:
nifty_close_df = nifty_df.selectExpr('Date','Close as nclose')
nifty_close_df.show(5)

+-----------+--------+
|       Date|  nclose|
+-----------+--------+
|29-Sep-2023| 19638.3|
|28-Sep-2023|19523.55|
|27-Sep-2023|19716.45|
|26-Sep-2023| 19664.7|
|25-Sep-2023|19674.55|
+-----------+--------+
only showing top 5 rows



In [14]:
join_df = fodf_nifty.join(nifty_close_df, (upper(fodf_nifty['Timestamp']) == upper(nifty_close_df['Date'])), 'inner')
join_df.show(5)

+----------+------+-----------+---------+----------+--------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+-----------+-------+
|INSTRUMENT|SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|    OPEN|    HIGH|     LOW|  CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|       Date| nclose|
+----------+------+-----------+---------+----------+--------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+-----------+-------+
|    FUTIDX| NIFTY|27-Jul-2023|      0.0|        XX| 19840.2|19872.35| 19600.0|19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|27-Jul-2023|19659.9|
|    FUTIDX| NIFTY|31-Aug-2023|      0.0|        XX| 19951.1| 19988.4|19751.05|19811.4|  19811.4|   185376| 1840717.8|11728350|  3456250|27-JUL-2023|27-Jul-2023|19659.9|
|    FUTIDX| NIFTY|28-Sep-2023|      0.0|        XX|20055.25|20092.85| 19876.9|19929.7|  19929.7|    18721| 186838.82| 1591800|   386150|27-JUL-2023|2

In [15]:
def mnameToNo(dt):
    mname = dt[3:6].upper()
    calendar = {"JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
                "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08", "SEP": "09", "OCT": "10",
                "NOV": "11", "DEC": "12"}
    return dt.upper().replace(mname, calendar[mname])

# create a udf from the month name to no function
# to apply to timestamp, expiry date colukns
udf_mname_to_no = udf(mnameToNo)

spark.udf.register('udf_mname_to_no', mnameToNo)

# put sale steps above
ssa = 10
# put buy steps below
bsb = -5


In [16]:
join_df.selectExpr("*",
"date(to_timestamp(udf_mname_to_no(timestamp), 'dd-MM-yyyy')) as tsdc",
"date(to_timestamp(udf_mname_to_no(expiry_dt), 'dd-MM-yyyy')) as expdc",
"date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as date",
"floor(nclose/50)*50 as sprnrcl").\
selectExpr("*", "dayOfWeek(tsdc) as dow").show(2)

+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+-----------+-------+----------+----------+----------+-------+---+
|INSTRUMENT|SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|    HIGH|     LOW|  CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|       Date| nclose|      tsdc|     expdc|      date|sprnrcl|dow|
+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+-----------+-------+----------+----------+----------+-------+---+
|    FUTIDX| NIFTY|27-Jul-2023|      0.0|        XX|19840.2|19872.35| 19600.0|19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|27-Jul-2023|19659.9|2023-07-27|2023-07-27|2023-07-27|  19650|  5|
|    FUTIDX| NIFTY|31-Aug-2023|      0.0|        XX|19951.1| 19988.4|19751.05|19811.4|  19811.4|   185376| 1840717.8|11728350|  3456250|27-JUL-2

In [17]:
join_df_ext = join_df.filter("option_typ = 'PE'").selectExpr("*",
"date(to_timestamp(udf_mname_to_no(timestamp), 'dd-MM-yyyy')) as tsdc",
"date(to_timestamp(udf_mname_to_no(expiry_dt), 'dd-MM-yyyy')) as expdc",
"date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as ndate",
"floor(nclose/50)*50 as sprnrcl").drop("date").withColumnRenamed("ndate", "date").\
selectExpr("*", "sprnrcl + {} * 50 as psl".format(ssa),
           "sprnrcl + {} * 50 as pbl".format(bsb),
           "dayOfWeek(tsdc) as dow").cache()

In [18]:
join_df_ext.filter("tsdc = '2023-07-27'").select("expdc", "psl", "pbl").distinct().show()

+----------+-----+-----+
|     expdc|  psl|  pbl|
+----------+-----+-----+
|2023-12-28|20150|19400|
|2023-08-10|20150|19400|
|2023-08-03|20150|19400|
|2024-03-28|20150|19400|
|2023-07-27|20150|19400|
|2023-08-31|20150|19400|
|2027-06-24|20150|19400|
|2024-12-26|20150|19400|
|2025-12-24|20150|19400|
|2024-06-27|20150|19400|
|2025-06-26|20150|19400|
|2023-08-17|20150|19400|
|2026-12-31|20150|19400|
|2023-09-28|20150|19400|
|2026-06-25|20150|19400|
|2023-08-24|20150|19400|
|2028-06-29|20150|19400|
|2027-12-30|20150|19400|
+----------+-----+-----+



In [19]:
join_df_ext.filter("""
option_typ = 'PE' and strike_pr = 20150 and tsdc = '2023-07-27' and expdc = '2023-08-10'
""").show()

+----------+------+-----------+---------+----------+-----+-----+-----+-----+---------+---------+----------+--------+---------+-----------+-------+----------+----------+----------+-------+-----+-----+---+
|INSTRUMENT|SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP| OPEN| HIGH|  LOW|CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP| nclose|      tsdc|     expdc|      date|sprnrcl|  psl|  pbl|dow|
+----------+------+-----------+---------+----------+-----+-----+-----+-----+---------+---------+----------+--------+---------+-----------+-------+----------+----------+----------+-------+-----+-----+---+
|    OPTIDX| NIFTY|10-Aug-2023|  20150.0|        PE|309.6|383.9|309.6|383.9|    529.5|       10|    102.45|     200|      200|27-JUL-2023|19659.9|2023-07-27|2023-08-10|2023-07-27|  19650|20150|19400|  5|
+----------+------+-----------+---------+----------+-----+-----+-----+-----+---------+---------+----------+--------+---------+-----------+-------+----------+----------+----------+-----

In [20]:
exp_df= join_df_ext.selectExpr('date', 'nclose').where("tsdc=expdc").distinct()

In [21]:
exp_df.show(2)

+----------+-------+
|      date| nclose|
+----------+-------+
|2023-08-24|19386.7|
|2023-07-27|19659.9|
+----------+-------+
only showing top 2 rows



In [22]:
join_df_ext_pes =\
join_df_ext.filter("option_typ = 'PE' and (strike_pr = psl or strike_pr = pbl)")\
.selectExpr('tsdc', 'expdc', 'option_typ', 'strike_pr','{} as ssa'.format(ssa),
            'psl','{} as bsb'.format(bsb), 'pbl', 'close', 'nclose').cache()

In [23]:
join_df_ext_pes.show(2)

+----------+----------+----------+---------+---+-----+---+-----+-----+-------+
|      tsdc|     expdc|option_typ|strike_pr|ssa|  psl|bsb|  pbl|close| nclose|
+----------+----------+----------+---------+---+-----+---+-----+-----+-------+
|2023-07-27|2023-07-27|        PE|  19400.0| 10|20150| -5|19400| 0.15|19659.9|
|2023-07-27|2023-07-27|        PE|  20150.0| 10|20150| -5|19400|512.4|19659.9|
+----------+----------+----------+---------+---+-----+---+-----+-----+-------+
only showing top 2 rows



In [24]:
sdf = join_df_ext_pes.filter("strike_pr = psl").selectExpr("*", "close as pspr")
bdf = join_df_ext_pes.filter("strike_pr = pbl").selectExpr("tsdc as btsdc", "expdc as bexpdc",
                                                           "close as pbpr")

In [25]:
# get the put sale price
sdf.join(
    bdf,
    (sdf['tsdc'] == bdf['btsdc']) &
    (sdf['expdc'] == bdf['bexpdc']) &
    (sdf['pbl'] == sdf['pbl'])
).filter("tsdc = '2023-07-07'").show()

+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+
|      tsdc|     expdc|option_typ|strike_pr|ssa|  psl|bsb|  pbl| close| nclose|  pspr|     btsdc|    bexpdc|  pbpr|
+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+
|2023-07-07|2023-07-13|        PE|  19800.0| 10|19800| -5|19050| 456.7|19331.8| 456.7|2023-07-07|2023-07-13|  13.2|
|2023-07-07|2023-07-20|        PE|  19800.0| 10|19800| -5|19050| 436.8|19331.8| 436.8|2023-07-07|2023-07-20| 44.45|
|2023-07-07|2023-07-27|        PE|  19800.0| 10|19800| -5|19050|440.45|19331.8|440.45|2023-07-07|2023-07-27|  77.0|
|2023-07-07|2023-08-03|        PE|  19800.0| 10|19800| -5|19050|1002.4|19331.8|1002.4|2023-07-07|2023-08-03|101.35|
|2023-07-07|2023-08-10|        PE|  19800.0| 10|19800| -5|19050| 493.8|19331.8| 493.8|2023-07-07|2023-08-10| 173.3|
|2023-07-07|2023-08-31|        PE|  19800.0| 10|19800| -5|19050| 490.0|1

In [26]:
joined_pe_df = sdf.join(
    bdf,
    (sdf['tsdc'] == bdf['btsdc']) &
    (sdf['expdc'] == bdf['bexpdc']) &
    (sdf['pbl'] == sdf['pbl'])
).cache()

In [27]:
joined_pe_df.show(2)

+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+----+
|      tsdc|     expdc|option_typ|strike_pr|ssa|  psl|bsb|  pbl| close| nclose|  pspr|     btsdc|    bexpdc|pbpr|
+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+----+
|2023-07-27|2023-07-27|        PE|  20150.0| 10|20150| -5|19400| 512.4|19659.9| 512.4|2023-07-27|2023-07-27|0.15|
|2023-07-27|2023-08-03|        PE|  20150.0| 10|20150| -5|19400|431.35|19659.9|431.35|2023-07-27|2023-08-03|22.5|
+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+----+
only showing top 2 rows



In [28]:
nifty_df.show(2)

+-----------+-------+--------+--------+--------+
|       Date|   Open|    High|     Low|   Close|
+-----------+-------+--------+--------+--------+
|29-Sep-2023|19581.2|19726.25|19551.05| 19638.3|
|28-Sep-2023|19761.8|19766.65| 19492.1|19523.55|
+-----------+-------+--------+--------+--------+
only showing top 2 rows



In [29]:
nifty_df.selectExpr("date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as tsp").show(2)

+----------+
|       tsp|
+----------+
|2023-09-29|
|2023-09-28|
+----------+
only showing top 2 rows



In [30]:
nifty_df_wtsp = nifty_df.selectExpr("date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as tsp",
                                    "close as expclosingpr")

In [31]:
from pyspark.sql.functions import *
joined_pe_df.alias('jdf').join(
    nifty_df_wtsp.alias('ndf'),
    col('jdf.expdc') == col('ndf.tsp')
).show()

+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+----------+------------+
|      tsdc|     expdc|option_typ|strike_pr|ssa|  psl|bsb|  pbl| close| nclose|  pspr|     btsdc|    bexpdc|  pbpr|       tsp|expclosingpr|
+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+----------+------------+
|2023-07-27|2023-07-27|        PE|  20150.0| 10|20150| -5|19400| 512.4|19659.9| 512.4|2023-07-27|2023-07-27|  0.15|2023-07-27|     19659.9|
|2023-07-27|2023-08-03|        PE|  20150.0| 10|20150| -5|19400|431.35|19659.9|431.35|2023-07-27|2023-08-03|  22.5|2023-08-03|    19381.65|
|2023-07-27|2023-08-10|        PE|  20150.0| 10|20150| -5|19400| 383.9|19659.9| 383.9|2023-07-27|2023-08-10|  49.6|2023-08-10|     19543.1|
|2023-07-27|2023-08-17|        PE|  20150.0| 10|20150| -5|19400|772.25|19659.9|772.25|2023-07-27|2023-08-17| 327.8|2023-08-17|    19365.25|
|2023-07-27|2023-08-

In [44]:
fodf_nifty.show(2)

+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
|INSTRUMENT|SYMBOL|  EXPIRY_DT|STRIKE_PR|OPTION_TYP|   OPEN|    HIGH|     LOW|  CLOSE|SETTLE_PR|CONTRACTS|VAL_INLAKH|OPEN_INT|CHG_IN_OI|  TIMESTAMP|
+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
|    FUTIDX| NIFTY|27-Jul-2023|      0.0|        XX|19840.2|19872.35| 19600.0|19646.9|  19659.9|   137340|1354819.67| 2547150| -2361800|27-JUL-2023|
|    FUTIDX| NIFTY|31-Aug-2023|      0.0|        XX|19951.1| 19988.4|19751.05|19811.4|  19811.4|   185376| 1840717.8|11728350|  3456250|27-JUL-2023|
+----------+------+-----------+---------+----------+-------+--------+--------+-------+---------+---------+----------+--------+---------+-----------+
only showing top 2 rows



In [72]:
def generate_df_for_bps(symbol, ssa, bsb, fodf, nifty_df):
  fodf_nifty = fodf.filter("SYMBOL= '{}'".format(symbol))
  nifty_df_wtsp = nifty_df.selectExpr("date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as tsp",
                                    "close as expclosingpr")
  join_df_ext = join_df.filter("option_typ = 'PE'").selectExpr("*",
"date(to_timestamp(udf_mname_to_no(timestamp), 'dd-MM-yyyy')) as tsdc",
"date(to_timestamp(udf_mname_to_no(expiry_dt), 'dd-MM-yyyy')) as expdc",
"date(to_timestamp(udf_mname_to_no(date), 'dd-MM-yyyy')) as ndate",
"floor(nclose/50)*50 as sprnrcl").drop("date").withColumnRenamed("ndate", "date").\
selectExpr("*", "sprnrcl + {} * 50 as psl".format(ssa),
           "sprnrcl + {} * 50 as pbl".format(bsb),
           "dayOfWeek(tsdc) as dow").cache()
  join_df_ext_pes =\
join_df_ext.filter("option_typ = 'PE' and (strike_pr = psl or strike_pr = pbl)")\
.selectExpr('tsdc', 'expdc', 'option_typ', 'strike_pr','{} as ssa'.format(ssa),
            'psl','{} as bsb'.format(bsb), 'pbl', 'close', 'nclose').cache()
  sdf = join_df_ext_pes.filter("strike_pr = psl").selectExpr("*", "close as pspr")
  bdf = join_df_ext_pes.filter("strike_pr = pbl").selectExpr("tsdc as btsdc", "expdc as bexpdc",
                                                           "close as pbpr")
  joined_pe_df = sdf.join(
    bdf,
    (sdf['tsdc'] == bdf['btsdc']) &
    (sdf['expdc'] == bdf['bexpdc']) &
    (sdf['pbl'] == sdf['pbl'])
).cache()
  final_df_to_return = joined_pe_df.alias('jdf').join(
    nifty_df_wtsp.alias('ndf'),
    col('jdf.expdc') == col('ndf.tsp')
)
  return final_df_to_return

In [73]:
generate_df_for_bps('NIFTY', 10, -5, fodf, nifty_df).show()

+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+----------+------------+
|      tsdc|     expdc|option_typ|strike_pr|ssa|  psl|bsb|  pbl| close| nclose|  pspr|     btsdc|    bexpdc|  pbpr|       tsp|expclosingpr|
+----------+----------+----------+---------+---+-----+---+-----+------+-------+------+----------+----------+------+----------+------------+
|2023-07-27|2023-07-27|        PE|  20150.0| 10|20150| -5|19400| 512.4|19659.9| 512.4|2023-07-27|2023-07-27|  0.15|2023-07-27|     19659.9|
|2023-07-27|2023-08-03|        PE|  20150.0| 10|20150| -5|19400|431.35|19659.9|431.35|2023-07-27|2023-08-03|  22.5|2023-08-03|    19381.65|
|2023-07-27|2023-08-10|        PE|  20150.0| 10|20150| -5|19400| 383.9|19659.9| 383.9|2023-07-27|2023-08-10|  49.6|2023-08-10|     19543.1|
|2023-07-27|2023-08-17|        PE|  20150.0| 10|20150| -5|19400|772.25|19659.9|772.25|2023-07-27|2023-08-17| 327.8|2023-08-17|    19365.25|
|2023-07-27|2023-08-

In [75]:
write_location_for_ssa_bsb = '/content/drive/MyDrive/bps_output'

In [76]:
generate_df_for_bps('NIFTY', 10, -5, fodf, nifty_df).write\
.partitionBy('ssa', 'bsb').mode('overwrite').parquet(write_location_for_ssa_bsb)

In [77]:
spark.read.parquet(write_location_for_ssa_bsb).count()

448

In [78]:
generate_df_for_bps('NIFTY', 10, -5, fodf, nifty_df).count()

448